In [1]:
!pip install surprise

     |████████████████████████████████| 11.8MB 274kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670952 sha256=ef9e7311beb3c8c80b9af1a2f29ab06999020fadec41560ebd9e414a17dc1291
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [2]:
!pip install implicit

     |████████████████████████████████| 1.1MB 3.1MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp36-cp36m-linux_x86_64.whl size=3419457 sha256=a4dc6f173d972928d82f060bf87a36e172e8a90f5290e0cf6a882e592ed1f86f
  Stored in directory: /root/.cache/pip/wheels/bf/d4/ec/fd4f622fcbefb7521f149905295b2c26adecb23af38aa28217
Successfully built implicit


**Import libraries, Read the data¶**


In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import time
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, SlopeOne, NMF, SVDpp, KNNBaseline, CoClustering, accuracy
from surprise.model_selection import cross_validate
from __future__ import (absolute_import, division, print_function,unicode_literals)
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import sys
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler
import implicit


In [4]:
from google.colab import files
uploaded = files.upload()

Saving hotel_data_final.csv to hotel_data_final.csv


In [5]:
import io
df = pd.read_csv(io.BytesIO(uploaded['hotel_data_final.csv']))

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
# Explore Data
type(df)
df.size
df.shape
df.head()
df.tail()

pandas.core.frame.DataFrame

280728

(23394, 12)

,Hotel_id,Hotel,Address,City,Title,Users,Reviews,Rating,Hotel_Type,Price,Hotel_Category,User_id
0,0,La Quinta,5820 Walden Rd,Beaumont,This hotel is a dump! Upon ch...,margierodriguez.mcqueen,This hotel is a dump! Upon checking in was tol...,1.0,NaN,NaN,"Lodging,Motels,Hotels,Meeting & Event Planning...",15400
1,1,The Point,Po Box 1327,Saranac Lake,Great family time,motomomMaryland,The point is possibly one of our favorite rest...,5.0,NaN,NaN,"Hotels,Hotel,Restaurants",15740
2,1,The Point,Po Box 1327,Saranac Lake,Throw back to the Gatsby Era - don't miss The ...,ellen00007,From the minute you arrive until the minute yo...,5.0,NaN,NaN,"Hotels,Hotel,Restaurants",13760
3,1,The Point,Po Box 1327,Saranac Lake,The Point of it all,Szerzek,That is the name on one of their boats. And th...,5.0,NaN,NaN,"Hotels,Hotel,Restaurants",11027
4,1,The Point,Po Box 1327,Saranac Lake,The most amazing week end!,Sarahbnyc83,I was lucky enough to spend 2 nights at The Po...,5.0,NaN,NaN,"Hotels,Hotel,Restaurants",10306


,Hotel_id,Hotel,Address,City,Title,Users,Reviews,Rating,Hotel_Type,Price,Hotel_Category,User_id
23389,3128,Polynesian Plaza,2131 Kalakaua Ave,Honolulu,IN SEVERE NEED OF UPDATE - OVERHAUL,fordpipefitter,We spent 2 weeks in this hotel due to the cost...,2.0,2.0,Rs.17718,"Vacation Rentals,Resorts & Vacation Cottages,V...",13911
23390,3129,Quality Inn,2120 Jameson Pl S W,Decatur,NaN,RABBITS,VERY FRIENDLY AND GREAT PRICE. I HAVE STAYED A...,5.0,2.0,Rs.7373,"Lodges,Hotels,Hotels & Motels,Lodging,Motels",9306
23391,3130,Quality Inn,2120 Jameson Pl S W,Decatur,NaN,Contributor,Editorial Review by Citysearch Editors Colonia...,5.0,2.0,Rs.7373,"Lodges,Hotels,Hotels & Motels,Lodging,Motels",2888
23392,3130,Quality Inn,12439 Northwest Fwy,Houston,NaN,Houstontraveler,What Happened to this Place!!!! I come this ho...,NaN,2.0,Rs.7373,"Hotel & Motel Reservations,Hotels,Hotels & Mot...",4949
23393,3130,Quality Inn,12439 Northwest Fwy,Houston,NaN,Robyn Galbos,Budget-minded travelers appreciate amenities l...,NaN,2.0,Rs.7373,"Hotel & Motel Reservations,Hotels,Hotels & Mot...",9807


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23394 entries, 0 to 23393
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Hotel_id        23394 non-null  int64  
 1   Hotel           23394 non-null  object 
 2   Address         23394 non-null  object 
 3   City            23394 non-null  object 
 4   Title           23342 non-null  object 
 5   Users           23394 non-null  object 
 6   Reviews         23393 non-null  object 
 7   Rating          23365 non-null  float64
 8   Hotel_Type      22431 non-null  float64
 9   Price           21880 non-null  object 
 10  Hotel_Category  23394 non-null  object 
 11  User_id         23394 non-null  int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 2.1+ MB


In [9]:
# Checking for missing values
pd.isnull(df).any()
pd.isnull(df).sum()

Hotel_id          False
Hotel             False
Address           False
City              False
Title              True
Users             False
Reviews            True
Rating             True
Hotel_Type         True
Price              True
Hotel_Category    False
User_id           False
dtype: bool

Hotel_id             0
Hotel                0
Address              0
City                 0
Title               52
Users                0
Reviews              1
Rating              29
Hotel_Type         963
Price             1514
Hotel_Category       0
User_id              0
dtype: int64

In [10]:
# Dropping the missing values
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)

Shape of dataset  (21769, 12)
Index(['Hotel_id', 'Hotel', 'Address', 'City', 'Title', 'Users', 'Reviews',
       'Rating', 'Hotel_Type', 'Price', 'Hotel_Category', 'User_id'],
      dtype='object')


**Train-Test Split**

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(df[['User_id','Hotel_id']],df['Rating'],test_size=0.25, 
                                                  random_state=0)

y_train=pd.DataFrame(y_train)
train=X_train.join(y_train)

y_test=pd.DataFrame(y_test)
test=X_test.join(y_test)

In [12]:
X_train.shape
X_test.shape
y_train.shape
y_test.shape

(16326, 2)

(5443, 2)

(16326, 1)

(5443, 1)

**Collaborative Filtering**

Collaborative Filtering is based on the idea that users similar to a me can be used to predict how much I will like a particular product or service those users have used/experienced but I have not.

**1- Simple Collaborative filtering base on mean**

Recommend based on other user's rating histories.

In [13]:
def collab_mean(user_id, hotel_id):
    user_condition = X_train.Hotel_id == hotel_id
    return y_train[user_condition].mean()

i=30
j=96
print('Collaborative mean filter for userId:',i, 'and hotelid:', j, 'is:',collab_mean(i,j).values)
print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
print('')

i=3529
j=25
print('Collaborative mean filter for userId:',i, 'and hotelid:', j, 'is:',collab_mean(i,j).values)
print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
print('')



i=13532
j=9
print('Collaborative mean filter for userId:',i, 'and hotelid:', j, 'is:',collab_mean(i,j).values)
print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
print('')

Collaborative mean filter for userId: 30 and hotelid: 96 is: [4.17241379]
Actual rating value is: 5.0

Collaborative mean filter for userId: 3529 and hotelid: 25 is: [4.29545455]
Actual rating value is: 4.0

Collaborative mean filter for userId: 13532 and hotelid: 9 is: [4.12299465]
Actual rating value is: 4.0



**2- Matrix Factorization-based algorithms**

The idea is basically to take a large (or potentially huge) matrix and factor it into some smaller representation of the original matrix. You can think of it in the same way as we would take a large number and factor it into two much smaller primes. We end up with two or more lower dimensional matrices whose product equals the original one. When we talk about collaborative filtering for recommender systems we want to solve the problem of our original matrix having millions of different dimensions, but our “tastes” not being nearly as complex. Even if i’ve viewed hundreds of items they might just express a couple of different tastes. Here we can actually use matrix factorization to mathematically reduce the dimensionality of our original “all users by all items” matrix into something much smaller that represents “all items by some taste dimensions” and “all users by some taste dimensions”. These dimensions are called latent or hidden features and we learn them from our data. Doing this reduction and working with fewer dimensions makes it both much more computationally efficient and but also gives us better results since we can reason about items in this more compact “taste space”. If we can express each user as a vector of their taste values, and at the same time express each item as a vector of what tastes they represent. You can see we can quite easily make a recommendation. This also gives us the ability to find connections between users who have no specific items in common but share common tastes.

**SVD**

SVD is a matrix factorization technique that is usually used to reduce the number of features of a data set by reducing space dimensions from N to K where K < N. For the purpose of the recommendation systems however, we are only interested in the matrix factorization part keeping same dimensionality. The matrix factorization is done on the user-item ratings matrix.

I have used Surprise library which has several powerful algorithms like Singular Value Decomposition (SVD), Non-negative Matrix Factorization (NMF), K Nearest Neighbor (KNN), and CoClusternig to minimise RMSE (Root Mean Square Error) and give recommendations.

In [14]:
data = Dataset.load_from_df(df[['User_id', 'Hotel_id', 'Rating']], Reader())


In [15]:
Models=[SVD(), NMF(), SlopeOne(), KNNBaseline(), CoClustering()]

for i in Models:
    start = time.time()
    cross_validate(i, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
    end = time.time()
    print('Total Elapsed time with model is:',end - start)
    print(' ')
    print(' ')

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0920  1.0927  1.0686  1.0628  1.0534  1.0739  0.0159  
MAE (testset)     0.8409  0.8455  0.8343  0.8284  0.8189  0.8336  0.0094  
Fit time          1.01    1.00    1.03    0.99    1.00    1.01    0.01    
Test time         0.09    0.02    0.02    0.02    0.02    0.04    0.03    


{'fit_time': (1.0145986080169678,
  0.99989914894104,
  1.0349128246307373,
  0.9949736595153809,
  1.0029783248901367),
 'test_mae': array([0.84093383, 0.84551416, 0.83429655, 0.82838328, 0.8189338 ]),
 'test_rmse': array([1.092049  , 1.09272692, 1.06859826, 1.06275815, 1.05337493]),
 'test_time': (0.09206318855285645,
  0.02463841438293457,
  0.022702693939208984,
  0.02418231964111328,
  0.021952390670776367)}

Total Elapsed time with model is: 5.4676361083984375
 
 
Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2075  1.2058  1.1934  1.2110  1.2276  1.2091  0.0110  
MAE (testset)     0.9447  0.9327  0.9293  0.9425  0.9523  0.9403  0.0083  
Fit time          1.82    1.77    1.80    1.83    1.78    1.80    0.02    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    


{'fit_time': (1.8184788227081299,
  1.7737064361572266,
  1.8031120300292969,
  1.8278958797454834,
  1.7829198837280273),
 'test_mae': array([0.94465416, 0.93268978, 0.92927805, 0.94253046, 0.95227211]),
 'test_rmse': array([1.20749341, 1.20580774, 1.19341235, 1.21098642, 1.22763133]),
 'test_time': (0.021968364715576172,
  0.02250838279724121,
  0.021834850311279297,
  0.022576332092285156,
  0.023462772369384766)}

Total Elapsed time with model is: 9.429572582244873
 
 
Evaluating RMSE, MAE of algorithm SlopeOne on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2140  1.2229  1.2031  1.1854  1.1957  1.2042  0.0132  
MAE (testset)     0.9278  0.9287  0.9253  0.8991  0.9082  0.9178  0.0120  
Fit time          0.24    0.24    0.23    0.22    0.23    0.23    0.01    
Test time         0.04    0.04    0.04    0.04    0.04    0.04    0.00    


...

Total Elapsed time with model is: 1.5841078758239746
 
 
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0696  1.0882  1.0877  1.0998  1.1011  1.0893  0.0113  
MAE (testset)     0.8359  0.8439  0.8462  0.8489  0.8504  0.8451  0.0051  
Fit time          6.73    6.17    6.29    5.48    5.78    6.09    0.43    
Test time         0.11    0.10    0.10    0.10    0.10    0.1

{'fit_time': (6.725710868835449,
  6.168901443481445,
  6.293074131011963,
  5.480642318725586,
  5.77620792388916),
 'test_mae': array([0.83594969, 0.84387283, 0.84623635, 0.84891598, 0.8503965 ]),
 'test_rmse': array([1.06957134, 1.08819495, 1.08767866, 1.09981903, 1.1011415 ]),
 'test_time': (0.10628533363342285,
  0.09845876693725586,
  0.10327410697937012,
  0.09610176086425781,
  0.10003972053527832)}

Total Elapsed time with model is: 31.27657198905945
 
 
Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2092  1.2042  1.1898  1.1838  1.1853  1.1945  0.0103  
MAE (testset)     0.9320  0.9228  0.9108  0.9097  0.9123  0.9175  0.0086  
Fit time          1.64    1.50    1.33    1.33    1.56    1.47    0.12    
Test time         0.02    0.02    0.02    0.02    0.02    0.02    0.00    


{'fit_time': (1.6386120319366455,
  1.4955706596374512,
  1.3326516151428223,
  1.3339905738830566,
  1.5636346340179443),
 'test_mae': array([0.93200796, 0.92282337, 0.91077171, 0.90972795, 0.91229453]),
 'test_rmse': array([1.20921993, 1.20416384, 1.18975802, 1.18379986, 1.18531494]),
 'test_time': (0.02033829689025879,
  0.017812490463256836,
  0.01880359649658203,
  0.01885199546813965,
  0.018313884735107422)}

Total Elapsed time with model is: 7.713924884796143
 
 


**Comparing the predicted rating values of some samples with their actual values**

In [16]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Models=[SVD(), NMF(), SlopeOne(), KNNBaseline(), CoClustering()]
m=['SVD','NMF','Slop One','KNN Baseline', 'CoClustering']

count=0

for algo in Models:
    print(m[count])
    start = time.time()
    algo.fit(trainset)
    predictions = algo.test(testset)
    end = time.time()    
    print('Total Elapsed time with model is:',round(end - start,3)) 
    i=30
    j=96
    print('Prediction of a particular user:', algo.predict(i,j, 3).est)
    print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
    print('')
    i=3529
    j=25
    print('Prediction of a particular user:', algo.predict(i,j, 3).est)
    print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
    print('')
    i=13532
    j=9
    print('Prediction of a particular user:', algo.predict(i,j, 3).est)
    print('Actual rating value is:', df[(df.User_id==i)&(df.Hotel_id==j)]['Rating'].mean())
    print('')
    print('')
    count+=1

SVD


Total Elapsed time with model is: 1.02
Prediction of a particular user: 4.298323785507733
Actual rating value is: 5.0

Prediction of a particular user: 4.018955593009876
Actual rating value is: 4.0

Prediction of a particular user: 4.136046598712923
Actual rating value is: 4.0


NMF


Total Elapsed time with model is: 1.82
Prediction of a particular user: 4.772582293345962
Actual rating value is: 5.0

Prediction of a particular user: 3.9540126686545762
Actual rating value is: 4.0

Prediction of a particular user: 4.040855077790028
Actual rating value is: 4.0


Slop One


Total Elapsed time with model is: 0.266
Prediction of a particular user: 5
Actual rating value is: 5.0

Prediction of a particular user: 4.0
Actual rating value is: 4.0

Prediction of a particular user: 4.040855077790028
Actual rating value is: 4.0


KNN Baseline
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


Total Elapsed time with model is: 5.164
Prediction of a particular user: 5
Actual rating value is: 5.0

Prediction of a particular user: 4.416942455987391
Actual rating value is: 4.0

Prediction of a particular user: 4.147751189956168
Actual rating value is: 4.0


CoClustering


Total Elapsed time with model is: 1.474
Prediction of a particular user: 5
Actual rating value is: 5.0

Prediction of a particular user: 4.995706249814135
Actual rating value is: 4.0

Prediction of a particular user: 4.040855077790028
Actual rating value is: 4.0




**3- Alternating Least Squares (ALS) Collaborative Filtering**

Alternating Least Squares (ALS) is a the model we’ll use to fit our data and find similarities. ALS is an iterative optimization process where we for every iteration try to arrive closer and closer to a factorized representation of our original data. We have our original matrix R of size u x i with our users, items and some type of feedback data. We then want to find a way to turn that into one matrix with users and hidden features of size u x f and one with items and hidden features of size f x i. In U and V we have weights for how each user/item relates to each feature. What we do is we calculate U and V so that their product approximates R as closely as possible: R ≈ U x V.

By randomly assigning the values in U and V and using least squares iteratively we can arrive at what weights yield the best approximation of R. The least squares approach in it’s basic forms means fitting some line to the data, measuring the sum of squared distances from all points to the line and trying to get an optimal fit by minimising this value.

With the alternating least squares approach we use the same idea but iteratively alternate between optimizing U and fixing V and vice versa. We do this for each iteration to arrive closer to R = U x V.

In [17]:
data=df
sparse_item_user = sparse.csr_matrix((data['Rating'].astype(float), (data['Hotel_id'], data['User_id'])))
sparse_user_item = sparse.csr_matrix((data['Rating'].astype(float), (data['Hotel_id'], data['User_id'])))

# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

model.fit(data_conf)

GPU training requires factor size to be a multiple of 32. Increasing factors from 20 to 32.


**Finding Hotels Similar to Best Western Lamplighter Inn Suites at SDSU:**

In [18]:
#---------------------
# FIND SIMILAR HOTELS
#---------------------
Hotel_id = 318 #Best Western Lamplighter Inn Suites at SDSU
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(Hotel_id, n_similar)

a=[]
b=[]
c=[]
# Print the names of our most similar movies
for item in similar:
    idx, score = item
    a.append(data.Hotel_id.loc[data.Hotel_id == idx].iloc[0])
    b.append(score)
    recommendations = pd.DataFrame({'HotelId': a, 'Score': b})

for i in a:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])
recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,318,1.000000,Best Western Lamplighter Inn Suites at SDSU
1,1049,0.999636,Hampton Inn Myrtle Beach-Northwood
2,1045,0.838414,Courtyard Macon
3,1324,0.763371,"The Westin Poinsett, Greenville"
4,2292,0.759369,Home2 Suites By Hilton Greenville Airport
5,2894,0.717386,Americas Best Value Inn
6,414,0.685309,Residence Inn Springfield South
7,431,0.648663,Hilton Garden Inn Atlanta Midtown
8,518,0.610695,Century Hotel
9,2323,0.547980,Wingate by Wyndham Lake Charles Casino Area


**Finding Hotels Similar to Nob Hill Hotel:**

In [19]:
#---------------------
# FIND SIMILAR HOTELS
#---------------------
Hotel_id = 296 #Nob Hill Hotel
n_similar = 10

# Use implicit to get similar items.
similar = model.similar_items(Hotel_id, n_similar)

a=[]
b=[]
c=[]
# Print the names of our most similar movies
for item in similar:
    idx, score = item
    a.append(data.Hotel_id.loc[data.Hotel_id == idx].iloc[0])
    b.append(score)
    recommendations = pd.DataFrame({'HotelId': a, 'Score': b})

for i in a:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])

recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,296,1.000000,Nob Hill Hotel
1,713,0.966372,Barn Motor Inn
2,1291,0.966313,Magnuson Grand Hotel
3,2937,0.719617,Hilton Garden Inn San Diego Rancho Bernardo
4,2493,0.598173,Comfort Inn
5,2404,0.573330,Red Roof Inn Hardeeville
6,721,0.571488,La Quinta Inn & Suites Warner Robins - Robins AFB
7,3034,0.557285,Grand Mercure Bangalore
8,1115,0.550481,Quality Inn at Collins Road
9,116,0.540938,Best Western Sandman Motel


**Recommend Hotels to User_id = 1:**

In [20]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 1
User_id =1

# Use the implicit recommender.
recommended = model.recommend(User_id, sparse_user_item)

hotels = []
scores = []
c=[]

for item in recommended:
    idx, score=item
    hotels.append(data.Hotel_id.loc[data.Hotel_id==idx].iloc[0])
    scores.append(score)


recommendations = pd.DataFrame({'HotelId': hotels, 'Score': scores})

for i in hotels:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])
recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,3099,0.665039,Shangri-La Hotel Bengaluru
1,3036,0.539164,Conrad Bengaluru
2,3070,0.533975,Taj West End
3,866,0.531525,Estancia La Jolla Hotel and Spa
4,2664,0.471684,Drury Inn Suites Colorado Springs Near The Air...
5,3072,0.465013,"The Oberoi, Bengaluru"
6,2684,0.458930,"Mandarin Oriental, Miami"
7,3084,0.415754,The Lalit Ashok
8,3068,0.415576,The Park Bangalore
9,3081,0.415047,Hotel Royal Orchid Regenta Bangalore


**Recommend Hotels to User_id = 75:**

In [21]:
#------------------------------
# CREATE USER RECOMMENDATIONS
#------------------------------

# Create recommendations for user with id 75
User_id =75

# Use the implicit recommender.
recommended = model.recommend(User_id, sparse_user_item)

hotels = []
scores = []
c=[]

for item in recommended:
    idx, score=item
    hotels.append(data.Hotel_id.loc[data.Hotel_id==idx].iloc[0])
    scores.append(score)


recommendations = pd.DataFrame({'HotelId': hotels, 'Score': scores})

for i in hotels:
    c.append(df[df.Hotel_id==i].Hotel.iloc[0])
    
recommendations['Hotel']=c
recommendations

,HotelId,Score,Hotel
0,90,0.893425,Orchid Key Inn
1,38,0.349527,Doubletree By Hilton Hotel Bay City - Riverfront
2,2569,0.303266,Hotel Zephyr
3,2929,0.296106,Hampton Inn San Diego Del Mar
4,2791,0.291370,Hilton Garden Inn Orlando Airport
5,1775,0.286435,Conrad Chicago
6,1549,0.278724,Stone Brook Resort
7,316,0.271280,Ocean Park Inn
8,656,0.263420,Hyatt House Seattle/Downtown
9,2943,0.261725,Courtyard by Marriott Philadelphia Downtown
